In [18]:
from google.colab import drive
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler

pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.max_columns", None)
pd.set_option("display.min_rows", 50)
pd.set_option("display.max_rows", 50)
pd.set_option("display.width", None)

drive_path = "/content/drive"
drive.mount(drive_path)
data_path = f"{drive_path}/My Drive/Colab Notebooks/gnn/input/march-machine-learning-mania-2025"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
res = pd.DataFrame()

for type_ in ["RegularSeason", "NCAATourney"]:
  res = pd.concat([res, pd.read_csv(f"{data_path}/M{type_}DetailedResults.csv")])

res = res.reset_index(drop=True)

res.insert(0, "WID", res["Season"].astype(str) + "_" + res["DayNum"].astype(str).str.zfill(3) + "_" + res["WTeamID"].astype(str) + "_" + res["LTeamID"].astype(str))
res.loc[res["WTeamID"] < res["LTeamID"], "WID"] = res["Season"].astype(str) + "_" + res["DayNum"].astype(str).str.zfill(3) + "_" + res["LTeamID"].astype(str) + "_" + res["WTeamID"].astype(str)
res.insert(1, "LID", res["WID"])

res.insert(2, "WSeason", res.pop("Season"))
res.insert(3, "LSeason", res["WSeason"])

res.insert(4, "WDayNum", res.pop("DayNum"))
res.insert(5, "LDayNum", res["WDayNum"])

res.insert(7, "LTeamID", res.pop("LTeamID"))

res.insert(8, "WOppID", res["LTeamID"])
res.insert(9, "LOppID", res["WTeamID"])

res.insert(10, "WMargin", res["WScore"] - res["LScore"])
res.insert(11, "LMargin", -res["WMargin"])

res.insert(12, "WLoc", res.pop("WLoc").map({"A": -1, "N": 0, "H": 1}))
res.insert(13, "LLoc", -res["WLoc"])

res.insert(14, "WNumOT", res.pop("NumOT"))
res.insert(15, "LNumOT", res["WNumOT"])

res = pd.concat([
    res[[c for c in res if c[0] == "W"]].rename(columns={c: c[1:] for c in res if c[0] == "W"}),
    res[[c for c in res if c[0] == "L"]].rename(columns={c: c[1:] for c in res if c[0] == "L"}),
]).sort_values("ID").reset_index(drop=True)

for c in res.loc[:, "Season":"OppID"]:
  res[c] = res[c].astype("int32")

sy = StandardScaler()
cy = "Margin"
res[cy] = sy.fit_transform(res[[cy]]).astype("float32")

cf = "Loc"
for c in res.loc[:, cf:]:
  res[c] = res[c].astype("float32")

res.loc[:, cf:] = StandardScaler().fit_transform(res.loc[:, cf:])

print(f"res {res.shape}")
print(res)
print()
res.info()

res (240528, 22)
                        ID  Season  DayNum  TeamID  OppID    Margin       Loc     NumOT     Score       FGM       FGA      FGM3      FGA3       FTM       FTA        OR        DR       Ast        TO       Stl       Blk        PF
0       2003_010_1328_1104    2003      10    1104   1328  0.397053  0.000000 -0.225257 -0.153137  0.490802  0.226886 -1.252690 -0.951573 -0.476284 -0.233940  0.862797  0.073893 -0.012935  2.304793  0.173707 -1.023865  0.846855
1       2003_010_1328_1104    2003      10    1328   1104 -0.397053  0.000000 -0.225257 -0.634959 -0.537384 -0.438796 -1.583592 -1.611310  0.346381  0.273258 -0.095526 -0.317623 -1.148182  1.129768  0.848504 -0.583745  0.399831
2       2003_010_1393_1272    2003      10    1393   1272 -0.463229  0.000000 -0.225257 -0.554655 -0.126110  1.425113 -0.259983  0.697769 -0.805351  0.019659  2.300281  0.269650 -1.375231 -0.280261  0.511106  1.176735 -0.494217
3       2003_010_1393_1272    2003      10    1272   1393  0.463229  0.

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tensor_kwargs = dict(dtype=torch.float32, device=device)
x = torch.tensor(res.loc[:, "Loc":].values, **tensor_kwargs)
print(f"x {x.shape}")
print(x)

x torch.Size([240528, 16])
tensor([[ 0.0000, -0.2253, -0.1531,  ...,  0.1737, -1.0239,  0.8469],
        [ 0.0000, -0.2253,  0.0075,  ..., -0.8385,  0.2965, -0.0472],
        [ 0.0000, -0.2253, -0.6350,  ...,  0.8485, -0.5837,  0.3998],
        ...,
        [ 0.0000, -0.2253, -0.1531,  ...,  0.1737,  2.4971,  0.3998],
        [ 0.0000, -0.2253, -1.3577,  ..., -1.1759, -0.5837, -0.9412],
        [ 0.0000, -0.2253,  1.6135,  ..., -0.5011, -1.4640, -0.2707]])
